In [102]:
pip install transformers

In [2]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

--2022-04-06 05:20:24--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M   198MB/s    in 0.2s    

2022-04-06 05:20:24 (198 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2022-04-06 05:20:24--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[======

In [52]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [53]:
# KorQuAD
# def read_squad(path):
#     path = Path(path)
#     with open(path, 'rb') as f:
#         squad_dict = json.load(f)
# 
#     contexts = []
#     questions = []
#     answers = []
#     for group in squad_dict['data']:
#         for passage in group['paragraphs']:
#             context = passage['context']
#             for qa in passage['qas']:
#                 question = qa['question']
#                 for answer in qa['answers']:
#                     contexts.append(context)
#                     questions.append(question)
#                     answers.append(answer)

#     return contexts, questions, answers

# train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
# val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [54]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                answer = qa['answers']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('train.json')

In [113]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    guids = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                guid = qa['guid']
                contexts.append(context)
                questions.append(question)
                guids.append(guid)

    return contexts, questions, guids

test_contexts, test_questions, test_guids = read_squad('test.json')

In [115]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(test_contexts))
print('테스트 데이터의 질문 개수 :', len(test_questions))
print('테스트 데이터의 아이디 개수 :', len(test_guids))

훈련 데이터의 본문 개수 : 17663
훈련 데이터의 질문 개수 : 17663
훈련 데이터의 답변 개수 : 17663
테스트 데이터의 본문 개수 : 4008
테스트 데이터의 질문 개수 : 4008
테스트 데이터의 아이디 개수 : 4008


In [116]:
print('첫번째 샘플의 본문')
print('-----------------')
print(train_contexts[0])

첫번째 샘플의 본문
-----------------
올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.


In [58]:
print('첫번째 샘플의 질문')
print('-----------------')
print(train_questions[0])

첫번째 샘플의 질문
-----------------
북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?


In [59]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[0])

첫번째 샘플의 답변
-----------------
{'text': '한 달가량', 'answer_start': 478}


In [60]:
print('첫번째 테스트트 데이터터 본문')
print('-----------------')
print(test_contexts[0])

첫번째 테스트트 데이터터 본문
-----------------
BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다.


In [61]:
print('첫번째 테스트트 데이터터 질문')
print('-----------------')
print(test_questions[0])

첫번째 테스트트 데이터터 질문
-----------------
말라카이트에서 나온 색깔을 사용한 에디션은?


In [117]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip()
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)
        assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러가 있습니다."
        answer['answer_end'] = end_idx

add_end_idx(train_answers, train_contexts)

In [63]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[1])

첫번째 샘플의 답변
-----------------
{'text': '한 달', 'answer_start': 478, 'answer_end': 481}


In [64]:
train_contexts[0][478]

'한'

In [65]:
train_contexts[0][479]

' '

In [66]:
train_contexts[0][480]

'달'

In [67]:
train_contexts[0][481]

'가'

In [68]:
train_contexts[0][482]

'량'

In [69]:
train_contexts[0][478:483]

'한 달가량'

In [121]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
test_encodings = tokenizer(test_contexts, test_questions, truncation=True, padding=True)

In [122]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    deleting_list = []

    for i in tqdm(range(len(answers))):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
              deleting_list.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list

In [123]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)

100%|██████████| 17663/17663 [00:00<00:00, 166414.69it/s]


In [124]:
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [125]:
def for_test(encodings):
  input_ids = np.array(encodings['input_ids'])
  attention_masks = np.array(encodings['attention_mask'])

  X_data = [input_ids, attention_masks]

  return X_data

In [126]:
X_test = for_test(test_encodings)

In [127]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)

In [77]:
# TPU 작동을 위한 코드 TPU 작동을 위한 코드
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimentalnnect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_s_coystem(resolver)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.0.169.218:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.0.169.218:8470


INFO:tensorflow:Finished initializing TPU system.


INFO:tensorflow:Finished initializing TPU system.


In [78]:
strategy = tf.distribute.TPUStrategy(resolver)

INFO:tensorflow:Found TPU system:


INFO:tensorflow:Found TPU system:


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Cores: 8


INFO:tensorflow:*** Num TPU Workers: 1


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [79]:
from transformers import TFBertModel

In [80]:
class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)
    
        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = tf.split(logits, 2, axis=-1)

        # start_logits = (batch_size, sequence_length,)
        # end_logits = (batch_size, sequence_length,)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

In [81]:
# 신규
with strategy.scope():
  model = TFBertForQuestionAnswering("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [82]:
history = model.fit(
    X_train,
    y_train,
    epochs=3,
    verbose=1,
    batch_size=16,
)

Epoch 1/3


1013/1013 [==============================] - 165s 97ms/step - loss: 3.3091 - output_1_loss: 1.5374 - output_2_loss: 1.7717
Epoch 2/3
1013/1013 [==============================] - 98s 97ms/step - loss: 1.7999 - output_1_loss: 0.7914 - output_2_loss: 1.0085
Epoch 3/3
1013/1013 [==============================] - 98s 97ms/step - loss: 1.3613 - output_1_loss: 0.5816 - output_2_loss: 0.7798


In [83]:
# 예측하는 함수
def predict_test_data_by_idx(idx):
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)
  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1]))+1
  answer_encoded = X_test[0][idx][start:end]
  print('예측 :',tokenizer.decode(answer_encoded))
  print('----------------------------------------')

In [128]:
tokenizer.decode(X_test[0][1]).split('[SEP] ')[0]

'[CLS] 프랑스 남부 알프스에 떨어져 150명의 사망자를 낸 저먼윙스 여객기는 부조종사가 의도적으로 추락시킨 것으로 보인다고 프랑스 검찰이 26일 발표했다. AFP통신 등에 따르면 사건을 조사 중인 프랑스 검찰의 브리스 로뱅 검사는 이날 기자회견을 열어 “ 조종실 음성녹음장치를 확인한 결과 조종사가 조종실 밖으로 나간 사이 부조종사가 의도적으로 여객기를 파괴하려 한 것 같다 ” 고 밝혔다. 로뱅 검사는 “ 부조종사가 의도적으로 여객기 하강버튼을 눌렀고, 여객기의 하강속도가 빨라졌다 ” 며 “ 사고 직전 조종석 밖에 있던 조종사가 문을 여러 차례 두드리고 소리를 질렀지만, 부조종사는 문을 열지 않았다 ” 고 설명했다. 그는 “ 마지막 순간까지 부조종사의 호흡은 정상이었으며 조종실에서는 침묵이 흘렀다 ” 고 덧붙였다. 부조종사는 28세인 독일 국적의 안드레아스 루비츠로, 1년6개월 전 조종간을 잡기 시작했으며 비행시간 경력은 630시간인 것으로 파악됐다. 로뱅 검사는 “ 부조종사의 테러 연관성은 확인되지 않았다 ” 며 “ 테러 공격이라고 의심할 만한 이유는 없다 ” 고 말했다. 저먼윙스 여객기는 지난 24일 스페인 바르셀로나에서 독일 뒤셀도르프로 향하던 중 마지막 교신이 이뤄지고 나서 약 8분간 3만2000피트 ( 약 9754ｍ ) 를 급강하해 알프스 산을 들이받았다. 로뱅 검사는 “ 여객기가 추락할 때 프랑스 관제탑이 ‘ 메이데이 ’ 라는 조난신호를 받지 못했다 ” 고 말했다. 이 사고로 승객 144명과 승무원 6명 등 탑승자 150명이 모두 사망했다. '

In [84]:
for i in range(0, 100):
  predict_test_data_by_idx(i)

본문 : [CLS] BMW 코리아 ( 대표 한상윤 ) 는 창립 25주년을 기념하는 ‘ BMW 코리아 25주년 에디션 ’ 을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션 ( 이하 25주년 에디션 ) 은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린 ( 50대 한정 ) 또는 마카오 블루 ( 50대 한정 ) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션 ( 25대 한정 ) 은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션 ( 25대 한정 ) 은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션 ( 7대 한정 ) 에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션 ( 8대 한정 ) 은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있다. 
질문 : 말라카이트에서 나온 색깔을 사용한 에디션은? 
예측 

In [139]:
import csv
def save_csv(X_test):
    f = open("submission.csv", "w", encoding = 'utf8')
    ln = len(test_questions)
    for i in tqdm(range(0,ln)):
          output = model([tf.constant(X_test[0][i])[None, :], tf.constant(X_test[1][i])[None, :]])
          start = tf.math.argmax(tf.squeeze(output[0]))
          end = tf.math.argmax(tf.squeeze(output[1]))+1
          answer_encoded = X_test[0][i][start:end]
          f.write(str(i) + ',' + test_guids[i] + ',' + tokenizer.decode(answer_encoded)+'\n')
          question = tokenizer.decode(X_test[0][i]).split('[SEP]')[1]
          print(test_guids[i])
          print(question)
          print(tokenizer.decode(answer_encoded))
    f.close()
    return ln

save_csv(X_test)

  0%|          | 1/4008 [00:00<1:06:01,  1.01it/s]

d14cb73158624cf094c546d856fd3c80
말라카이트에서 나온 색깔을 사용한 에디션은? 
뉴 740Li 25주년 에디션


  0%|          | 2/4008 [00:01<1:06:33,  1.00it/s]

906631384e91493ebe1c7f34aea6f241
사고 비행기의 목적지는? 
독일 뒤셀도르프로


  0%|          | 3/4008 [00:02<1:06:00,  1.01it/s]

...

100%|█████████▉| 4007/4008 [1:06:07<00:00,  1.01it/s]

0a73550b36df4baf82ac2f98619d22e7
강남지역에 사는 학생들은 며칠 동안 학교를 안가나? [SEP]
사흘


100%|██████████| 4008/4008 [1:06:08<00:00,  1.01it/s]

dfe6ef25f84f461e89e54d370e4521d5
로이터통신에 따르면, EU인프라펀드 출자에 대한 중국의 공표가 있을 나라는? 



4008